In [1]:
# script to assemble mechanisms Frankenstein style
import os
import cantera as ct
import cantera.ck2yaml
import numpy as np
import pandas as pd
import concurrent.futures
import rmgpy.chemkin


import sys
sys.path.append(os.path.join(os.environ['AUTOSCIENCE_REPO'], 'analysis'))

import ignition_delay

import os
import re
import glob
import yaml

import rmgpy.chemkin
import rmgpy.data.kinetics
import rmgpy.data.thermo
import rmgpy.data.rmg


import numpy as np
import pandas as pd
# import importlib
# importlib.reload(rmgpy.data.kinetics)

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

import sys
sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/database/')
import database_fun
import subprocess

import matplotlib.pyplot as plt
%matplotlib inline

DFT_DIR = os.environ['DFT_DIR']

Loading DFT database from /work/westgroup/harris.se/autoscience/reaction_calculator/database


In [3]:
basedir = '/work/westgroup/harris.se/autoscience/concatenate'

# basedir = '/scratch/harris.se/guassian_scratch/RMG_min/RMG_min_6'
chemkin_file = os.path.join(basedir, 'chem_annotated.inp')
transport_file = os.path.join(basedir, 'tran.dat')
sp_dict = os.path.join(basedir, 'species_dictionary.txt')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, sp_dict, check_duplicates=False)

# chemkin2 = os.path.join(basedir, 'chemkin2.inp')
# rmgpy.chemkin.save_chemkin_file(chemkin2, species_list, reaction_list, check_for_duplicates=True)


In [3]:
# file_name

In [4]:
# file_name = os.path.join(basedir, os.path.splitext(os.path.basename(chemkin2))[0] + ".yaml")
# parser = ct.ck2yaml.Parser()
# result, other = parser.convert_mech(chemkin_file, transport_file=transport_file, out_name=file_name, quiet=False, permissive=False)

    

# Check for duplicate Arrhenius with harris_butane

In [3]:
for i in range(len(reaction_list)):
    if hasattr(reaction_list[i], 'family') and 'harris_butane' in reaction_list[i].family.lower():
        print(i, reaction_list[i])

33 HO2(16) + C[CH2](20) <=> H2O2(17) + C2H4(11)
34 OH(15) + butane(1) <=> H2O(8) + [CH2]CCC(24)
35 O2(2) + [CH2]CCC(24) <=> HO2(16) + C4H8(47)
36 O2(2) + C[CH]CC(23) <=> HO2(16) + C4H8(47)
37 OH(15) + C3H6(12) <=> H2O(8) + [CH2]C=C(22)
38 C[CH]CC(23) + CC(CCOO)OO(183) <=> C4H9O4(181) + butane(1)
39 HO2(16) + C[CH]CC(23) <=> O2(2) + butane(1)
40 H2O2(17) + C[CH]CC(23) <=> HO2(16) + butane(1)
41 CCOO(54) + C[CH]CC(23) <=> CCO[O](28) + butane(1)
42 C[CH]CC(23) + CCC(C)OO(80) <=> CCC(C)O[O](52) + butane(1)
43 H2O2(17) + [CH2]CCC(24) <=> HO2(16) + butane(1)
44 C[CH]CC(23) + CCCCOO(226) <=> CCCCO[O](215) + butane(1)
45 [CH2]CCC(24) + CCCCOO(226) <=> CCCCO[O](215) + butane(1)
46 HO2(16) + CCCCO[O](215) <=> O2(2) + CCCCOO(226)
47 COO(31) + C[CH]CC(23) <=> CO[O](30) + butane(1)


In [4]:
bad_duplicate_pairs = []

for i in range(len(reaction_list)):
    for j in range(i):
        if reaction_list[i].is_isomorphic(reaction_list[j]):
            if not (reaction_list[i].duplicate and reaction_list[j].duplicate):
                bad_duplicate_pairs.append([i, j])

In [5]:
bad_duplicate_pairs

[[42, 40],
 [162, 160],
 [169, 167],
 [210, 208],
 [346, 344],
 [368, 356],
 [484, 416],
 [489, 415],
 [565, 563],
 [589, 587],
 [733, 65],
 [780, 779],
 [786, 785],
 [900, 886],
 [924, 913],
 [1099, 1098],
 [1126, 909],
 [1449, 1434],
 [2255, 770],
 [2257, 773]]

In [6]:
remove_list = []
remove_list_items = []
for bad_duplicate_pair in bad_duplicate_pairs:
    i = bad_duplicate_pair[0]
    j = bad_duplicate_pair[1]
    
        
    # one PDEP and one library
    if type(reaction_list[i]) == rmgpy.rmg.pdep.PDepReaction and \
            type(reaction_list[j]) == rmgpy.data.kinetics.library.LibraryReaction:
        # mark them as duplicates
        reaction_list[i].duplicate = True
        reaction_list[j].duplicate = True
    elif type(reaction_list[i]) != rmgpy.data.kinetics.library.LibraryReaction and \
            type(reaction_list[j]) == rmgpy.rmg.pdep.PDepReaction:
        # mark them as duplicates
        reaction_list[i].duplicate = True
        reaction_list[j].duplicate = True
        
    # one library and one family
    elif type(reaction_list[i]) == rmgpy.data.kinetics.library.LibraryReaction and \
            type(reaction_list[j]) != rmgpy.data.kinetics.library.LibraryReaction:
        remove_list.append(j)
        remove_list_items.append(reaction_list[j])
    elif type(reaction_list[i]) != rmgpy.data.kinetics.library.LibraryReaction and \
            type(reaction_list[j]) == rmgpy.data.kinetics.library.LibraryReaction:
        remove_list.append(i)
        remove_list_items.append(reaction_list[i])
        
    # one PDEP and one family
    elif type(reaction_list[i]) == rmgpy.rmg.pdep.PDepReaction and \
            type(reaction_list[j]) != rmgpy.rmg.pdep.PDepReaction:
        # mark them as duplicates
        reaction_list[i].duplicate = True
        reaction_list[j].duplicate = True
    elif type(reaction_list[i]) != rmgpy.rmg.pdep.PDepReaction and \
            type(reaction_list[j]) == rmgpy.rmg.pdep.PDepReaction:
        # mark them as duplicates
        reaction_list[i].duplicate = True
        reaction_list[j].duplicate = True
        

    # two PDEPs
    elif type(reaction_list[i]) == rmgpy.rmg.pdep.PDepReaction and \
        type(reaction_list[j]) == rmgpy.rmg.pdep.PDepReaction:
        # remove the one with more reactants
        if len(reaction_list[i].reactants) > reaction_list[j].reactants:
            remove_list.append(i)
            remove_list_items.append(reaction_list[i])
        elif len(reaction_list[i].reactants) < reaction_list[j].reactants:
            remove_list.append(j)
            remove_list_items.append(reaction_list[j])
        else:  # remove j by default because it was added second
            remove_list.append(j)
            remove_list_items.append(reaction_list[j])
            
    else:
        print(type(reaction_list[i]))
        print(type(reaction_list[j]))
        print()

In [9]:
# remove_list = []
# remove_list_items = []
# for bad_duplicate_pair in bad_duplicate_pairs:
#     i = bad_duplicate_pair[0]
#     j = bad_duplicate_pair[1]
    
        
#     # one library and one family
#     if type(reaction_list[i]) == rmgpy.data.kinetics.library.LibraryReaction and \
#             type(reaction_list[j]) != rmgpy.data.kinetics.library.LibraryReaction:
#         remove_list.append(j)
#         remove_list_items.append(reaction_list[j])
#     elif type(reaction_list[i]) != rmgpy.data.kinetics.library.LibraryReaction and \
#             type(reaction_list[j]) == rmgpy.data.kinetics.library.LibraryReaction:
#         remove_list.append(i)
#         remove_list_items.append(reaction_list[i])
        
#     # one PDEP and one family
#     elif type(reaction_list[i]) == rmgpy.rmg.pdep.PDepReaction and \
#             type(reaction_list[j]) != rmgpy.rmg.pdep.PDepReaction:
#         # mark them as duplicates
#         reaction_list[i].duplicate = True
#         reaction_list[j].duplicate = True
#     elif type(reaction_list[i]) != rmgpy.rmg.pdep.PDepReaction and \
#             type(reaction_list[j]) == rmgpy.rmg.pdep.PDepReaction:
#         # mark them as duplicates
#         reaction_list[i].duplicate = True
#         reaction_list[j].duplicate = True
        

#     # two PDEPs
#     elif type(reaction_list[i]) == rmgpy.rmg.pdep.PDepReaction and \
#         type(reaction_list[j]) == rmgpy.rmg.pdep.PDepReaction:
#         # remove the one with more reactants
#         if len(reaction_list[i].reactants) > reaction_list[j].reactants:
#             remove_list.append(i)
#             remove_list_items.append(reaction_list[i])
#         elif len(reaction_list[i].reactants) < reaction_list[j].reactants:
#             remove_list.append(j)
#             remove_list_items.append(reaction_list[j])
#         else:  # remove j by default because it was added second
#             remove_list.append(j)
#             remove_list_items.append(reaction_list[j])
            
#     else:
#         print(type(reaction_list[i]))
#         print(type(reaction_list[j]))
#         print()

In [7]:
for item in remove_list_items:
    reaction_list.remove(item)

In [8]:
outfile = os.path.join(basedir, 'new_chemkin2.inp')

rmgpy.chemkin.save_chemkin_file(outfile, species_list, reaction_list)

In [47]:
remove_list_items

[TemplateReaction(index=46, reactants=[Species(index=16, label="HO2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[4.02956,-0.00263991,1.52232e-05,-1.71674e-08,6.26751e-12,322.677,4.84426], Tmin=(100,'K'), Tmax=(923.91,'K')), NASAPolynomial(coeffs=[4.15132,0.00191148,-4.11288e-07,6.34989e-11,-4.86412e-15,83.4261,3.09348], Tmin=(923.91,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2"""), molecule=[Molecule(smiles="[O]O")], molecular_weight=(33.0068,'amu')), Species(index=23, label="C[CH]CC", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.50678,0.0557587,-0.000247308,6.65302e-07,-5.84346e-10,5715.06,10.7682], Tmin=(10,'K'), Tmax=(408.79,'K')), NASAPolynomial(coeffs=[-0.395744,0.0445264,-2.47606e-05,6.65015e-09,-6.94851e-13,6447.04,31.1551], Tmin=(408.79,'K'), Tmax=(3000,'K'))], Tmin=(10,'K'), Tmax=(3000,'K'), comment="""Thermo library: harris_butane_0001"""), molecule=[Molecule(smiles="C[CH]CC")], molecular_weight=(57.1143,'amu'))], pr

In [46]:
remove_list

[45]

In [31]:
reaction_list[0].family

'BurkeH2O2inN2'

In [32]:
type(reaction_list[0])

rmgpy.data.kinetics.library.LibraryReaction

In [33]:
type(reaction_list[51])

rmgpy.data.kinetics.family.TemplateReaction

In [37]:
type(reaction_list[69])

rmgpy.rmg.pdep.PDepReaction

In [26]:
print(reaction_list[69])
print(reaction_list[51])

O2(2) + C[CH2](21) <=> HO2(16) + C2H4(11)
O2(2) + C[CH2](21) <=> HO2(16) + C2H4(11)


In [29]:
print(reaction_list[51].family)
print(reaction_list[69].family)

Disproportionation


AttributeError: 'PDepReaction' object has no attribute 'family'

In [23]:
bad_duplicate_pairs

[[45, 33],
 [69, 51],
 [77, 71],
 [235, 229],
 [244, 242],
 [314, 272],
 [581, 569],
 [590, 582],
 [860, 844],
 [895, 118],
 [926, 652],
 [975, 961],
 [1013, 1009],
 [1018, 1016],
 [1052, 1050],
 [1061, 1057],
 [1077, 1069],
 [1103, 1090],
 [1316, 1310],
 [1332, 983]]

In [22]:
remove_list

[45]

In [11]:
reaction_list[45].kinetics.comment

'Estimated using template [X_H;C_rad/H/NonDeC] for rate rule [Orad_O_H;C_rad/H/NonDeC]\nEuclidian distance = 2.0\nfamily: H_Abstraction'

In [17]:
reaction_list[33].family

'harris_butane_0001'

In [15]:
print(reaction_list[33])

HO2(16) + C[CH]CC(23) <=> O2(2) + butane(1)


In [16]:
database_fun.get_unique_reaction_index(reaction_list[33])

4721

In [7]:
reaction_list[10].duplicate

False